In [1]:
!pip install kagglehub

In [26]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer,Trainer,TrainingArguments
import kagglehub

In [27]:
train_data = kagglehub.load_dataset(
  kagglehub.KaggleDatasetAdapter.PANDAS,
  "nileshmalode1/samsum-dataset-text-summarization",
  "samsum-train.csv" # Changed the path argument to point to the 'test.csv' file within the Kaggle dataset.
)

validation_data = kagglehub.load_dataset(
  kagglehub.KaggleDatasetAdapter.PANDAS,
  "nileshmalode1/samsum-dataset-text-summarization",
  "samsum-validation.csv" # Changed the path argument to point to the 'test.csv' file within the Kaggle dataset.
)

train_data.head()

<ipython-input-27-d391f13b8161>:1: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  train_data = kagglehub.load_dataset(
<ipython-input-27-d391f13b8161>:7: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  validation_data = kagglehub.load_dataset(


,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [28]:
train_data = train_data.sample(n=7000, random_state=42).reset_index(drop=True)
validation_data = validation_data.sample(n=500, random_state=42).reset_index(drop=True)


In [29]:
import re

def clean_text(text):
  text = re.sub(r"\r\n", " ", text)
  text = re.sub(r"\s+"," ",text)
  text = re.sub(r"<.*?>"," ",text)
  text = text.strip().lower()
  return text

train_data["dialogue"] = train_data["dialogue"].apply(clean_text)
train_data["summary"] = train_data["summary"].apply(clean_text)

validation_data["dialogue"] = validation_data["dialogue"].apply(clean_text)
validation_data["summary"] = validation_data["summary"].apply(clean_text)

In [30]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

In [31]:
def preprocess_function(examples):
  input = tokenizer(examples["dialogue"],truncation=True,padding="max_length",max_length=512)
  target = tokenizer(examples["summary"],truncation=True,padding="max_length",max_length=512)
  input["labels"] = target["input_ids"]
  return input

train_data = train_data.apply(preprocess_function,axis=1)
validation_data = validation_data.apply(preprocess_function,axis=1)

In [32]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 6,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_steps=100,
)

trainer = Trainer(
    model = model,
    args= training_args,
    train_dataset = train_data,
    eval_dataset = validation_data
)

trainer.train()

In [34]:
model.save_pretrained("./text_summarizer")
tokenizer.save_pretrained("./text_summarizer")

('./text_summarizer/tokenizer_config.json',
 './text_summarizer/special_tokens_map.json',
 './text_summarizer/spiece.model',
 './text_summarizer/added_tokens.json')

In [ ]:
def generate_summary(text):
  text = clean_text(text)
  input_ids = tokenizer.encode(text,return_tensors="pt",truncation=True,padding="max_length",max_length=512)
  input_ids = input_ids.to(model.device)
  output = model.generate(input_ids,max_length=512,num_beams=8,early_stopping=True)
  output = tokenizer.decode(output[0],skip_special_tokens=True,)
  return output

while True:
  text = input("Enter paragraph here : ")
  if text == "exit":
    break
  print(generate_summary(text))

In [40]:
import shutil
from google.colab import files

model_dir = "./text_summarizer"

shutil.make_archive("text_summarizer", 'zip', model_dir)
files.download('text_summarizer.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>